In [58]:
# Importing Libraries
import pandas as pd
import numpy as np
from pathlib import Path

In [74]:
# Reading Data
data = pd.read_csv('data/news_headlines.csv')

In [75]:
data

,publish_date,headline_text
0,20030303,unhooked brakes to blame for taiwan train disa...
1,20030918,oldest prisoner in tas released citing health
2,20030913,nine reportedly dead in portuguese plane crash
3,20031031,nurses welcome medicare rebate plan
4,20030930,un cuts its iraq staff
5,20030601,tougher water restrictions loom after dry may
6,20030715,illegal fishing review to hold public meetings
7,20030723,minardi sign italian bruni as test driver
8,20030910,man breaks leg in cliff fall
9,20031125,bush signs off on troops pay rise


In [77]:
initial_year = 2003
final_year = 2017

years = np.arange(initial_year,final_year+1)

for year in years:
    data['publish_date']//10000 == year
    print(data['publish_date']//10000)


0          2003
1          2003
2          2003
3          2003
4          2003
5          2003
6          2003
7          2003
8          2003
9          2003
10         2003
11         2003
12         2003
13         2003
14         2003
15         2003
16         2003
17         2003
18         2003
19         2003
20         2003
21         2003
22         2003
23         2003
24         2003
25         2003
26         2003
27         2003
28         2003
29         2003
           ... 
999971     2017
999972     2017
999973     2017
999974     2017
999975     2017
999976     2017
999977     2017
999978     2017
999979     2017
999980     2017
999981     2017
999982     2017
999983     2017
999984     2017
999985     2017
999986     2017
999987     2017
999988     2017
999989     2017
999990     2017
999991     2017
999992     2017
999993     2017
999994     2017
999995     2017
999996     2017
999997     2017
999998     2017
999999     2017
1000000    2017
Name: publish_date, Leng

0          2003
1          2003
2          2003
3          2003
4          2003
5          2003
6          2003
7          2003
8          2003
9          2003
10         2003
11         2003
12         2003
13         2003
14         2003
15         2003
16         2003
17         2003
18         2003
19         2003
20         2003
21         2003
22         2003
23         2003
24         2003
25         2003
26         2003
27         2003
28         2003
29         2003
           ... 
999971     2017
999972     2017
999973     2017
999974     2017
999975     2017
999976     2017
999977     2017
999978     2017
999979     2017
999980     2017
999981     2017
999982     2017
999983     2017
999984     2017
999985     2017
999986     2017
999987     2017
999988     2017
999989     2017
999990     2017
999991     2017
999992     2017
999993     2017
999994     2017
999995     2017
999996     2017
999997     2017
999998     2017
999999     2017
1000000    2017
Name: publish_date, Leng

In [21]:
np.where(data.isnull(data))

TypeError: isnull() takes 1 positional argument but 2 were given